In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')

# 高速公路网的预处理

In [ ]:
data = gpd.read_file(r'gis/广东省高速公路_简化.json')
# 提取简化高速公路边信息
edge = data[['geometry']]
edge['edge_id'] = range(len(edge))

# 生成反向边信息
edge_inverse = edge.copy()
from shapely.geometry import LineString
edge_inverse['geometry'] = edge_inverse['geometry'].apply(lambda x:LineString(list(x.coords)[::-1]))
edge_inverse['edge_id'] +=len(edge)

# 合并正反向边信息
edge = pd.concat([edge,edge_inverse])

In [ ]:
# 收费站、服务区的额外节点加入
station = gpd.read_file(r'gis/收费站测试/收费站.shp')
station['id'] = range(len(station))
# 投影、偏移、匹配至最近的边
dist = 50
# 偏移100米的边
edge_offset = edge.copy()
edge_offset.crs = 'EPSG:4326'
edge_offset_proj = edge_offset.to_crs('EPSG:4525') #CGCS2000 / 3-degree Gauss Kruger zone 37
edge_offset_proj['geometry'] = edge_offset_proj['geometry'].apply(lambda x:x.parallel_offset(dist))
edge_offset_proj = edge_offset_proj.reset_index(drop=True)

station.crs = 'EPSG:4326'
station_proj = station.to_crs('EPSG:4525')

station_proj_match = tbd.ckdnearest_line(station_proj,edge_offset_proj[-edge_offset_proj['geometry'].is_empty][['edge_id','geometry']])
matched_table = station_proj_match[['id','edge_id']]


In [ ]:
# 基于额外节点调整网络
for i in range(len(matched_table)):
    r = matched_table.iloc[i]
    nodeid = r['id']
    edgeid = r['edge_id']
    nodegeometry = station[station['id']==nodeid]['geometry'].iloc[0]
    edgegeometry = edge[edge['edge_id']==edgeid]['geometry'].iloc[0]

    #找到edge上距离node最近的点
    projectdist = edgegeometry.project(nodegeometry)
    projectpoint = edgegeometry.interpolate(projectdist)

    #如果在两端，则不需要切分
    #增加端点到节点的线段
    add_edges = []
    if (projectdist==edgegeometry.length)|(projectdist==0):
        line3 = LineString([nodegeometry,projectpoint])
        line4 = LineString([projectpoint,nodegeometry])
        add_edges.append(line3)
        add_edges.append(line4)
    else: #如果在中间，则需要切分，将原始边切分为两段，再加上端点到节点的线段
        from shapely.geometry import Point
        edge_coords = pd.DataFrame(edgegeometry.coords)
        edge_coords['proj'] = edge_coords.apply(lambda r:edgegeometry.project(Point([r[0],r[1]])),axis = 1)
        # 由中间端点切分边为两段
        line1 = LineString(edge_coords[edge_coords['proj']<projectdist][[0,1]].apply(lambda r:Point([r[0],r[1]]),axis = 1).tolist()+[projectpoint])
        line2 = LineString([projectpoint] + edge_coords[edge_coords['proj']>projectdist][[0,1]].apply(lambda r:Point([r[0],r[1]]),axis = 1).tolist())
        # 添加中间端点到收费站的线段
        line3 = LineString([nodegeometry,projectpoint])
        line4 = LineString([projectpoint,nodegeometry])
        add_edges.append(line1)
        add_edges.append(line2)
        add_edges.append(line3)
        add_edges.append(line4)
        # 此时需要删除原有的边
        #edge = edge[edge['edge_id']!=edgeid]

    # 将新的边加入
    add_edges = gpd.GeoDataFrame(geometry=add_edges)
    edge = pd.concat([edge,add_edges])
    # 重新生成边的id
    edge['edge_id'] = range(len(edge))

In [ ]:
# 构建节点表
edge['slon'] = edge['geometry'].apply(lambda r:r.coords[0][0])
edge['slat'] = edge['geometry'].apply(lambda r:r.coords[0][1])
edge['elon'] = edge['geometry'].apply(lambda r:r.coords[-1][0])
edge['elat'] = edge['geometry'].apply(lambda r:r.coords[-1][1])

# 提取简化高速公路节点信息
node = pd.concat([edge[['slon','slat']].rename(columns = {'slon':'lon','slat':'lat'}),
                  edge[['elon','elat']].rename(columns = {'elon':'lon','elat':'lat'})]).drop_duplicates()
node['geometry'] = gpd.points_from_xy(node['lon'],node['lat'])
node['id'] = range(len(node)) 

# 生成收费站节点对应网络节点的编号
station2node = tbd.ckdnearest_point(station,node)[['id_x','id_y']].rename(columns = {'id_x':'station_id','id_y':'node_id'})

# 为边添加起终点接节点信息
## 添加起点信息
node_tmp = node[['lon','lat','id']]
node_tmp.columns = ['slon','slat','u']
edge = pd.merge(edge,node_tmp,on = ['slon','slat'],how = 'left')
## 添加终点信息
node_tmp = node[['lon','lat','id']]
node_tmp.columns = ['elon','elat','v']
edge = pd.merge(edge,node_tmp,on = ['elon','elat'],how = 'left')

In [ ]:
# 高速公路边向右平移一定距离形成面
def generate_plane(edge,dist = 100):
    '''
    高速公路边向右平移一定距离形成面

    '''
    ## 转换为投影坐标系
    edge.crs = 'EPSG:4326'
    edge['length'] = edge.to_crs('EPSG:4525').length
    edge_plane = edge.to_crs('EPSG:4525') #CGCS2000 / 3-degree Gauss Kruger zone 37
    edge['length'] = edge_plane['geometry'].length

    ## 生成单方向偏移
    from shapely.geometry import Polygon
    edge_plane['geometry'] = edge_plane['geometry'].apply(lambda x:Polygon(list(x.coords)+list(x.parallel_offset(dist).coords)))
    edge_plane = edge_plane.to_crs('EPSG:4326')
    return edge_plane

edge_plane = generate_plane(edge,dist = 100)

In [ ]:
#存储边、面与节点信息
edge.to_file(r'拓扑网络/广东省高速公路_简化_边.json',driver = 'GeoJSON')
edge_plane.to_file(r'拓扑网络/广东省高速公路_简化_面.json',driver = 'GeoJSON')
node = gpd.GeoDataFrame(node)
node.to_file(r'拓扑网络/广东省高速公路_简化_节点.json',driver = 'GeoJSON')
station2node.to_csv(r'拓扑网络/收费站节点对应网络节点的编号.csv',index = None)

# 高速公路网络建模

## 网络构建

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#存储边、面与节点信息
edge = gpd.read_file(r'拓扑网络/广东省高速公路_简化_边.json')
edge_plane = gpd.read_file(r'拓扑网络/广东省高速公路_简化_面.json')
node = gpd.read_file(r'拓扑网络/广东省高速公路_简化_节点.json')
station2node = pd.read_csv(r'拓扑网络/收费站节点对应网络节点的编号.csv')
# 将station2node转换为字典
station2node_dict = station2node.set_index('station_id')['node_id'].to_dict()


In [ ]:
#提取边与节点信息，构建网络
G_edges = edge[['u','v','length']].values.tolist()
G_nodes = list(node['id'])

import networkx as nx
#先创建一个有向图
G = nx.DiGraph()
#添加节点
G.add_nodes_from(G_nodes) 
#添加边
G.add_weighted_edges_from(G_edges)

## OD出行路径信息提取

In [ ]:
## OD表构建
o = station2node[['station_id']]
o['flag'] = 1
d = o.copy()
od = pd.merge(o,d,on = 'flag')[['station_id_x','station_id_y']]
od = od[od['station_id_x']!=od['station_id_y']]

In [ ]:
# 获取出行路径
def getshortestpath(r):
    # 获取出行路径
    start_node = station2node_dict[r['station_id_x']]
    end_node = station2node_dict[r['station_id_y']]
    shortest_path = nx.shortest_path(G, source=start_node, target=end_node)
    return list(map(int,shortest_path))# 转为整型

# 获取OD的出行路径
od['path'] = od.apply(lambda r:getshortestpath(r),axis = 1)
# 对OD编号
od['odid'] = range(len(od))
od.to_csv(r'拓扑网络/OD路径表.csv',index = None)

In [ ]:
od = pd.read_csv(r'拓扑网络/OD路径表.csv')

## OD所经过的路段信息提取

In [ ]:
def get_path_gdf(shortest_path,edge):
    '''
    从点序列中获取出行路段
    '''
    path = pd.DataFrame(shortest_path,columns=['u'])
    path['u'] = path['u'].astype(int)
    path['v'] = path['u'].shift(-1).fillna(0).astype(int)
    path = path.iloc[:-1]
    path = gpd.GeoDataFrame(path)
    path = pd.merge(path,edge)
    path['cumsumlength'] = path['length'].cumsum()
    return path
#path = get_path_gdf(shortest_path,edge)
def get_path_dis_table(r,edge):
    '''
    获取OD所经过的路段信息
    '''
    shortest_path = list(map(int,r['path'][1:-1].split(',')))
    path = get_path_gdf(shortest_path,edge)
    path = path[['edge_id','cumsumlength']]
    path['station_id_x'] = r['station_id_x']
    path['station_id_y'] = r['station_id_y']
    return path

#获取OD所经过的路段信息

od_dis_table = od.groupby('odid').apply(lambda r:get_path_dis_table(r.iloc[0],edge)).reset_index()[['station_id_x','station_id_y','edge_id','cumsumlength']]

# od_dis_table存储了每个OD对应的路段信息
# 其中，station_id_x、station_id_y为OD的起点、终点
# edge_id为经过路段的id，cumsumlength为经过路段的累计长度
# 例如，station_id_x=0，station_id_y=1，edge_id=47784，cumsumlength=68.596574，表示OD为0-1的出行路径，需要经过id为47784的路段，路径走完这一路段时，所经过的长度为68.596574米
od_dis_table.to_csv(r'拓扑网络/OD路段信息表.csv',index = None)

In [ ]:
start_coords = list(node[node['id'] == start_node][['lon','lat']].iloc[0])[::-1]
end_coords = list(node[node['id'] == end_node][['lon','lat']].iloc[0])[::-1]

import folium
from folium import GeoJson

m = folium.Map(location=[23.130196,113.259294],zoom_start=10, tiles="CartoDB Positron")

# 定义样式函数
def style_function_gray(feature):
    return {
        'fillColor': '#888',
        'color': '#888',
        'weight': 1,
        'fillOpacity': 0.5
    }
def style_function_red(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'weight': 1,
        'fillOpacity': 0.5
    }

GeoJson(edge_plane.to_json(), style_function=style_function_gray).add_to(m)

# 创建GeoJson对象，并使用样式函数
geojson = GeoJson(path.to_json(), style_function=style_function_red)

geojson.add_to(m)


# 在地图上添加起点和终点标记
folium.Marker(start_coords, tooltip="Start Point", icon=folium.Icon(color="green", icon="play")).add_to(m)
folium.Marker(end_coords, tooltip="End Point", icon=folium.Icon(color="red", icon="stop")).add_to(m)


m


In [ ]:
# OD对应的出行路径
# 生成OD矩阵